In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 날씨 데이터 불러오기

In [2]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv('./data/Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether

,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
일시,,,,,
2020-10-01 00:00:00,17.3,0.0,0.2,92.0,0.0
2020-10-01 01:00:00,16.8,0.0,0.5,94.0,0.0
2020-10-01 02:00:00,16.9,0.0,1.1,93.0,0.0
2020-10-01 03:00:00,16.8,0.0,0.4,94.0,0.0
2020-10-01 04:00:00,16.1,0.0,0.1,95.0,0.0
...,...,...,...,...,...
2022-10-01 19:00:00,21.1,0.0,0.1,54.0,0.0
2022-10-01 20:00:00,19.6,0.0,0.6,63.0,0.0
2022-10-01 21:00:00,18.8,0.0,0.7,73.0,0.0


# 차량 데이터 불러오기 및 전처리

# 한맥

장비별, 일시별 구분  
입차 대수 - 출차 대수 = 현재 차량 대수  
한맥의 경우 FilePath로부터 입출차 시간 추출  
정기권 등 구분 X

In [3]:
def pretreatment(df):
    df['일시'] = pd.to_datetime(df['FilePath'].apply(lambda x: x[12:26]), format='%Y%m%d%H%M%S')
    df['일시'] = pd.to_datetime(df['일시']).apply(del_min_sec)
    df = df[['일시', 'FilePath', 'InOutDiv']]
    df = df.rename({'FilePath':'주차일련번호'}, axis=1)
    return df

def del_min_sec(x):
    return x.replace(minute=0, second=0)

car_normal = pd.read_csv('./data/일반차량.csv')
car_normal = pretreatment(car_normal)

car_ticket = pd.read_csv('./data/정기권차량.csv')
car_ticket = pretreatment(car_ticket)

hanmeak = pd.concat([car_normal, car_ticket]).sort_values('일시')
hanmeak_in = hanmeak[hanmeak['InOutDiv']==1].iloc[:,:2]
hanmeak_in = hanmeak_in.groupby('일시').count().reset_index().rename({'주차일련번호':'입차대수'}, axis=1)

hanmeak_out = hanmeak[hanmeak['InOutDiv']==2].iloc[:,:2]
hanmeak_out = hanmeak_out.groupby('일시').count().reset_index().rename({'주차일련번호':'출차대수'}, axis=1)

hanmeak_all = pd.merge(hanmeak_in, hanmeak_out, on='일시', how='outer').fillna(0).sort_values('일시').reset_index(drop=True)
hanmeak_all['입출차 차이'] = hanmeak_all['입차대수'] - hanmeak_all['출차대수']
hanmeak_all['주차장'] = '주차1동, 주차2동'
hanmeak_all

,일시,입차대수,출차대수,입출차 차이,주차장
0,2020-10-01 04:00:00,1.0,0.0,1.0,"주차1동, 주차2동"
1,2020-10-01 05:00:00,0.0,1.0,-1.0,"주차1동, 주차2동"
2,2020-10-01 08:00:00,4.0,0.0,4.0,"주차1동, 주차2동"
3,2020-10-01 09:00:00,2.0,0.0,2.0,"주차1동, 주차2동"
4,2020-10-01 10:00:00,0.0,1.0,-1.0,"주차1동, 주차2동"
...,...,...,...,...,...
12312,2022-10-01 19:00:00,4.0,11.0,-7.0,"주차1동, 주차2동"
12313,2022-10-01 20:00:00,1.0,6.0,-5.0,"주차1동, 주차2동"
12314,2022-10-01 21:00:00,1.0,3.0,-2.0,"주차1동, 주차2동"
12315,2022-10-01 22:00:00,2.0,1.0,1.0,"주차1동, 주차2동"


# 다래

In [4]:
darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv('./data/VW_OutCarinfo (차량 출차 정보 조회).csv')
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_89021/1494210905.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')


## 이름 바꾸기 13_1동 지하 1 > 13_1동

In [5]:
import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0
for i in range(len(darae_out)):
    slicing(darae_out,i)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_89021/4221457405.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_89021/4221457405.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg


In [6]:
darae_in['입차시간'] = darae_in['입차일자'] +  darae_in['입차시각']
darae_out['출차시간'] = darae_out['출차일자']  + darae_out['출차시각']
darae_in['입차시간'] = pd.to_datetime(darae_in['입차시간']).apply(del_min_sec)
darae_out['출차시간'] = pd.to_datetime(darae_out['출차시간']).apply(del_min_sec)

darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

In [7]:
darae_in = darae_in.groupby(['입차시간','주차장']).count().reset_index().rename({'입차일자':'입차대수'}, axis=1).iloc[:,:3]
darae_out = darae_out.groupby(['출차시간','주차장']).count().reset_index().rename({'출차일자':'출차대수'}, axis=1).iloc[:,:3]

In [8]:
darae_out.head()

,출차시간,주차장,출차대수
0,2022-09-01 05:00:00,13-3동,2
1,2022-09-01 06:00:00,13-1동,2
2,2022-09-01 06:00:00,13-3동,5
3,2022-09-01 07:00:00,12동,1
4,2022-09-01 07:00:00,13-1동,1


## 빈 시간 채우기
- 처음에 9월 1일 0시 넣고 마지막에 9월 31일 23시 넣기

In [9]:
list(darae_out['주차장'].unique())

['13-3동', '13-1동', '12동', '13-2동', '14-1동', '15동', '14-2동']

In [10]:
def null_time(df,parking_name,time_column,InOut_column,start_time,end_time):
    df = df[df['주차장'] == parking_name]
    set_time = pd.date_range(start_time, end_time, freq='h')
    data =  pd.DataFrame(index=set_time)
    data = pd.merge(data, df, left_index=True, right_on=time_column, how='left')
    data['주차장'] = data['주차장'].fillna(parking_name)
    data[InOut_column] = data[InOut_column].fillna(0)
    return data

In [15]:
parking_list = list(darae_in['주차장'].unique())

darae_analysis = pd.DataFrame()
for i in parking_list:
    tmp_out = null_time(darae_out,i,'출차시간','출차대수','20220901000000','20220930235959').set_index('출차시간')
    tmp_in = null_time(darae_in,i,'입차시간','입차대수','20220901000000','20220930235959').set_index('입차시간')
    tmp_out = tmp_out.reset_index()
    tmp_in = tmp_in.reset_index()
    darae = pd.merge(tmp_in,tmp_out, left_on =['입차시간','주차장'], right_on = ['출차시간','주차장'] ,how = 'left')
    darae = darae.drop(['출차시간'],axis=1)
    darae = darae.rename({'입차시간' : '일시'},axis=1)
    darae['입출차 차이'] = darae['입차대수'] - darae['출차대수']
    darae['입출차 누계'] = darae['입출차 차이'].cumsum()
    darae = darae.set_index('일시')
    if darae['입출차 누계'].min() < 0:
        darae['입출차 누계'] -= darae['입출차 누계'].min()
    darae['연'] = darae.index.year
    darae['월'] = darae.index.month
    darae['일'] = darae.index.day
    darae['시각'] = darae.index.hour

    darae_analysis = pd.concat([darae_analysis,darae],axis=0)



darae_analysis

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,연,월,일,시각
일시,,,,,,,,,
2022-09-01 00:00:00,13-1동,0.0,0.0,0.0,0.0,2022,9,1,0
2022-09-01 01:00:00,13-1동,0.0,0.0,0.0,0.0,2022,9,1,1
2022-09-01 02:00:00,13-1동,0.0,0.0,0.0,0.0,2022,9,1,2
2022-09-01 03:00:00,13-1동,1.0,0.0,1.0,1.0,2022,9,1,3
2022-09-01 04:00:00,13-1동,0.0,0.0,0.0,1.0,2022,9,1,4
...,...,...,...,...,...,...,...,...,...
2022-09-30 19:00:00,14-1동,0.0,1.0,-1.0,0.0,2022,9,30,19
2022-09-30 20:00:00,14-1동,0.0,0.0,0.0,0.0,2022,9,30,20
2022-09-30 21:00:00,14-1동,0.0,0.0,0.0,0.0,2022,9,30,21


In [19]:
darae_analysis.columns

Index(['주차장', '입차대수', '출차대수', '입출차 차이', '입출차 누계', '연', '월', '일', '시각'], dtype='object')

In [22]:
from datetime import datetime
set_time = pd.date_range('20201001', '20221001230000', freq='h')
hanmeak_analysis = pd.DataFrame(index=set_time)
hanmeak_analysis = pd.merge(hanmeak_analysis, hanmeak_all, left_index=True, right_on='일시', how='left').fillna(0)
hanmeak_analysis = hanmeak_analysis.set_index('일시')
hanmeak_analysis['입출차 누계'] = hanmeak_analysis['입출차 차이'].cumsum()
# 누계가 음수인 경우, 최솟값만큼 기존에 차가 있었다고 가정함
if hanmeak_analysis['입출차 누계'].min() < 0:
    hanmeak_analysis['입출차 누계'] -= hanmeak_analysis['입출차 누계'].min()

hanmeak_analysis['연'] = hanmeak_analysis.index.year
hanmeak_analysis['월'] = hanmeak_analysis.index.month
hanmeak_analysis['일'] = hanmeak_analysis.index.day
hanmeak_analysis['시각'] = hanmeak_analysis.index.hour
hanmeak_analysis['주차장'] = hanmeak_analysis['주차장'].replace(0,'주차1동, 주차2동')
hanmeak_analysis = hanmeak_analysis[['주차장', '입차대수', '출차대수', '입출차 차이', '입출차 누계', '연', '월', '일', '시각']]
hanmeak_analysis

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,연,월,일,시각
일시,,,,,,,,,
2020-10-01 00:00:00,"주차1동, 주차2동",0.0,0.0,0.0,5.0,2020,10,1,0
2020-10-01 01:00:00,"주차1동, 주차2동",0.0,0.0,0.0,5.0,2020,10,1,1
2020-10-01 02:00:00,"주차1동, 주차2동",0.0,0.0,0.0,5.0,2020,10,1,2
2020-10-01 03:00:00,"주차1동, 주차2동",0.0,0.0,0.0,5.0,2020,10,1,3
2020-10-01 04:00:00,"주차1동, 주차2동",1.0,0.0,1.0,6.0,2020,10,1,4
...,...,...,...,...,...,...,...,...,...
2022-10-01 19:00:00,"주차1동, 주차2동",4.0,11.0,-7.0,166.0,2022,10,1,19
2022-10-01 20:00:00,"주차1동, 주차2동",1.0,6.0,-5.0,161.0,2022,10,1,20
2022-10-01 21:00:00,"주차1동, 주차2동",1.0,3.0,-2.0,159.0,2022,10,1,21


In [37]:
data = pd.concat([darae_analysis,hanmeak_analysis])
data = pd.merge(data, whether, left_index=True, right_index=True, how='left')
data.to_csv('주차예측(한맥+다래).csv')